In [1]:
# Obter o caminho para os notebooks e logs a partir das variáveis de ambiente
import os

user_name = os.environ.get('USER_NAME', 'spark')
user_home = os.environ.get('USER_HOME', '/home/spark')
account_key = os.environ.get('account_key')

In [2]:
pip install azure-storage-blob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 12.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient

In [4]:
# Construir a sessão Spark
spark = (SparkSession.builder 
    .appName("JupyterSparkExample") 
    .master("spark://spark-master:7077")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.4.0,com.microsoft.azure:azure-storage:8.6.4")
    .config("spark.hadoop.fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
    .config("spark.hadoop.fs.azure.account.key.rcblob01.blob.core.windows.net", account_key)
    .getOrCreate())

:: loading settings :: url = jar:file:/opt/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
org.apache.hadoop#hadoop-azure added as a dependency
com.microsoft.azure#azure-storage added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0a97dd8b-cec7-4aa0-8239-5edc0530038a;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-azure;3.4.0 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.2.0 in central
	found org.eclipse.jetty#jetty-util-ajax;9.4.53.v20231009 in central
	found org.eclipse.jetty#jetty-util;9.4.53.v20231009 in central
	found org.wildfly.openssl#wildfly-openssl;1.1.3.Final in central
	found com.microsoft.azure#azure-storage;8.6.4 in central
	found com.fasterxml.jackson.core#jackson-co

In [5]:
# Azure Blob Storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"

# Construct connection path
wasbs_path = f'wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/{blob_relative_path}'
print(wasbs_path)

# Read parquet data from Azure Blob Storage path
blob_df = spark.read.parquet(wasbs_path)
# blob_df.show()

wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow


24/07/30 01:19:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-azure-file-system.properties,hadoop-metrics2.properties


In [6]:
df = blob_df.select('vendorID').limit(10)

In [7]:
df.show()

+--------+
|vendorID|
+--------+
|     CMT|
|     VTS|
|     CMT|
|     CMT|
|     CMT|
|     VTS|
|     CMT|
|     VTS|
|     CMT|
|     VTS|
+--------+



In [8]:
root = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()).getWorkingDirectory()
print(root)

file:/home/spark/notebooks


In [10]:
from datetime import datetime

print(datetime.now())

2024-07-30 01:22:20.371649


In [11]:
(df.coalesce(2)
 .write
 .format("parquet")
 .mode("append")
 .save("wasbs://read-zone@rcblob01.blob.core.windows.net/silver_b"))


24/07/30 01:46:00 WARN AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1


In [12]:
print(datetime.now())

2024-07-30 01:46:01.085507
